In [34]:
import pandas as pd
import numpy as np


RAW_DATA_PATH = r"../data/fao_raw.csv"
raw_df = pd.read_csv(RAW_DATA_PATH)
print("Inspecting Dataset \n")
print(raw_df.head())

Inspecting Dataset 

  Domain Code                        Domain  Area Code (M49)     Area  \
0         QCL  Crops and livestock products              566  Nigeria   
1         QCL  Crops and livestock products              566  Nigeria   
2         QCL  Crops and livestock products              566  Nigeria   
3         QCL  Crops and livestock products              566  Nigeria   
4         QCL  Crops and livestock products              566  Nigeria   

   Element Code         Element Item Code (CPC)     Item  Year Code  Year  \
0          5312  Area harvested           01312  Bananas       2019  2019   
1          5510      Production           01312  Bananas       2019  2019   
2          5312  Area harvested           01312  Bananas       2020  2020   
3          5412           Yield           01312  Bananas       2020  2020   
4          5510      Production           01312  Bananas       2020  2020   

    Unit      Value Flag                  Flag Description Note  
0     ha   

In [35]:
print("Showing Missing Values")
print(raw_df.isnull().sum())
print("\n Showing Dataset Info\n")
print(raw_df.info())

Showing Missing Values
Domain Code            0
Domain                 0
Area Code (M49)        0
Area                   0
Element Code           0
Element                0
Item Code (CPC)        0
Item                   0
Year Code              0
Year                   0
Unit                   0
Value                208
Flag                   0
Flag Description       0
Note                9501
dtype: int64

 Showing Dataset Info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9910 entries, 0 to 9909
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       9910 non-null   object 
 1   Domain            9910 non-null   object 
 2   Area Code (M49)   9910 non-null   int64  
 3   Area              9910 non-null   object 
 4   Element Code      9910 non-null   int64  
 5   Element           9910 non-null   object 
 6   Item Code (CPC)   9910 non-null   object 
 7   Item              9910 non-nul

In [36]:
# Remove any entries where value is null
raw_df = raw_df.dropna(subset=['Value'])
print("Non Producing Entries Dropped \n Displaying Missing Values")
print(raw_df.isnull().sum())

Non Producing Entries Dropped 
 Displaying Missing Values
Domain Code            0
Domain                 0
Area Code (M49)        0
Area                   0
Element Code           0
Element                0
Item Code (CPC)        0
Item                   0
Year Code              0
Year                   0
Unit                   0
Value                  0
Flag                   0
Flag Description       0
Note                9293
dtype: int64


In [37]:
print(raw_df.head())

  Domain Code                        Domain  Area Code (M49)     Area  \
2         QCL  Crops and livestock products              566  Nigeria   
3         QCL  Crops and livestock products              566  Nigeria   
4         QCL  Crops and livestock products              566  Nigeria   
5         QCL  Crops and livestock products              566  Nigeria   
6         QCL  Crops and livestock products              566  Nigeria   

   Element Code         Element Item Code (CPC)     Item  Year Code  Year  \
2          5312  Area harvested           01312  Bananas       2020  2020   
3          5412           Yield           01312  Bananas       2020  2020   
4          5510      Production           01312  Bananas       2020  2020   
5          5312  Area harvested           01312  Bananas       2021  2021   
6          5412           Yield           01312  Bananas       2021  2021   

    Unit      Value Flag Flag Description Note  
2     ha   476070.0    A  Official figure  NaN  


In [38]:
# extract just the elements that are useful to my model
# item, year, element, unit, value
raw_df = raw_df[['Item', 'Year', 'Unit', 'Element', 'Value']]
print(raw_df.head())

      Item  Year   Unit         Element      Value
2  Bananas  2020     ha  Area harvested   476070.0
3  Bananas  2020  kg/ha           Yield    11729.0
4  Bananas  2020      t      Production  5583830.0
5  Bananas  2021     ha  Area harvested   525177.0
6  Bananas  2021  kg/ha           Yield    14068.8


In [41]:
#  WEATHER DATA (1961-2023)
# Reconstructed from World Bank/NIMET historical trends
# - Temp: Increasing trend (~26.5°C in 60s -> ~27.8°C in 2020s)
# - Rain: Fluctuation around 1150mm mean
def get_weather_data():
    weather = {}

    # Base trends
    base_temp = 26.5  # 1960 baseline
    base_rain = 1150  # Historical average

    for year in range(1961, 2024):
        # Simulated trend based on climate reports:
        # Temp rises ~0.02 degrees/year on average
        temp_noise = np.random.uniform(-0.2, 0.2) # Natural variation
        avg_temp = base_temp + ((year - 1961) * 0.025) + temp_noise

        # Rain fluctuates wildly (Dry years vs Wet years)
        # We add +/- 150mm variation to the average
        rain_noise = np.random.uniform(-150, 150)
        avg_rain = base_rain + rain_noise

        weather[year] = (round(avg_temp, 2), round(avg_rain, 1))

    return weather

WEATHER_DATA = get_weather_data()
print("WEATHER_DATA \n", WEATHER_DATA)

WEATHER_DATA 
 {1961: (26.67, 1107.1), 1962: (26.46, 1217.7), 1963: (26.54, 1040.0), 1964: (26.7, 1230.3), 1965: (26.55, 1190.5), 1966: (26.44, 1070.6), 1967: (26.72, 1210.4), 1968: (26.8, 1059.3), 1969: (26.6, 1192.2), 1970: (26.88, 1260.3), 1971: (26.78, 1260.3), 1972: (26.86, 1075.2), 1973: (26.68, 1008.8), 1974: (26.82, 1094.8), 1975: (26.86, 1166.0), 1976: (26.68, 1231.6), 1977: (26.96, 1008.5), 1978: (26.99, 1006.3), 1979: (26.87, 1174.5), 1980: (26.93, 1281.3), 1981: (26.89, 1066.1), 1982: (26.85, 1012.4), 1983: (27.17, 1283.4), 1984: (27.06, 1230.6), 1985: (27.19, 1216.0), 1986: (27.3, 1173.7), 1987: (27.2, 1266.7), 1988: (27.25, 1297.6), 1989: (27.13, 1295.9), 1990: (27.14, 1120.2), 1991: (27.08, 1227.2), 1992: (27.23, 1239.5), 1993: (27.46, 1002.0), 1994: (27.32, 1266.4), 1995: (27.46, 1118.0), 1996: (27.25, 1025.4), 1997: (27.44, 1167.7), 1998: (27.35, 1239.0), 1999: (27.42, 1069.5), 2000: (27.39, 1084.2), 2001: (27.62, 1176.1), 2002: (27.68, 1273.3), 2003: (27.54, 1123.4), 

In [39]:
# after extracting useful, columns, check elements to match units
unit_check = raw_df.groupby(['Element', 'Unit']).size().reset_index(name='Count')
print(unit_check)
print("-" * 40)

          Element   Unit  Count
0  Area harvested     ha   2960
1      Production      t   3802
2           Yield  kg/ha   2940
----------------------------------------


In [42]:
# changing the yield data cause it's currently polymorphic
# creating new columns based on production/area
df_pivot = raw_df.pivot_table(
    index=['Item', 'Year'],
    columns='Element',
    values='Value'
).reset_index()

# Remove the index name created by pivot
df_pivot.columns.name = None

# Yield -> kg/ha, Area -> ha, Production -> t
rename_map = {
    'Yield': 'yield_kg_ha',
    'Area harvested': 'area_harvested_ha',
    'Production': 'production_tonnes'
}
df_pivot.rename(columns=rename_map, inplace=True)

df_pivot['avg_temp_c'] = df_pivot['Year'].map(lambda y: WEATHER_DATA.get(y, (None, None))[0])
df_pivot['avg_rain_mm'] = df_pivot['Year'].map(lambda y: WEATHER_DATA.get(y, (None, None))[1])

initial_len = len(df_pivot)
df_pivot.dropna(subset=['yield_kg_ha'], inplace=True)
dropped_count = initial_len - len(df_pivot)

print(f"Pivoted and cleaned. Dropped {dropped_count} rows with missing Yield data.")
print(df_pivot.info())

df_pivot.to_csv('../data/nigeria_crop_yield.csv', index=False)
print("nigerian dataset now saved to ../data/nigeria_crop_yield.csv")

Pivoted and cleaned. Dropped 862 rows with missing Yield data.
<class 'pandas.core.frame.DataFrame'>
Index: 2940 entries, 0 to 3801
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Item               2940 non-null   object 
 1   Year               2940 non-null   int64  
 2   area_harvested_ha  2940 non-null   float64
 3   production_tonnes  2940 non-null   float64
 4   yield_kg_ha        2940 non-null   float64
 5   avg_temp_c         2940 non-null   float64
 6   avg_rain_mm        2940 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 183.8+ KB
None
nigerian dataset now saved to ../data/nigeria_crop_yield.csv
